In [2]:
from src.dataloader import DataLoader

dataloader = DataLoader()
variables, A, B, L = dataloader.load()

In [3]:
from cvxopt import solvers

for used_vars in variables[:1]:
    used_const = [0, 1, 2] # weights upper, lower bound & return lower bound

    for v in used_vars:
        used_const.append(3 + v)
        used_const.append(3003 + v)

    # optimals = solvers.lp(L[used_vars], A[used_const, used_vars], B[used_const])
    optimals2 = solvers.lp(L[used_vars], A[used_const, used_vars], B[used_const], solver="glpk")

In [4]:
optimals2

{'status': 'optimal',
 'x': <1781x1 matrix, tc='d'>,
 's': <3565x1 matrix, tc='d'>,
 'y': <0x1 matrix, tc='d'>,
 'z': <3565x1 matrix, tc='d'>,
 'primal objective': -0.7916923330773643,
 'dual objective': -0.7916923330773643,
 'gap': -3.2452097072119725e-17,
 'relative gap': -4.0990793665989056e-17,
 'primal infeasibility': 6.67703919525431e-16,
 'dual infeasibility': 1.7809024548261769e-16,
 'primal slack': -1.7288687842453854e-15,
 'dual slack': -0.0,
 'residual as primal infeasibility certificate': None,
 'residual as dual infeasibility certificate': None}